In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

In [2]:
# Read the csv file and store it in the variable df. Then it will show the df to ensure the file was read properly.
df = pd.read_csv(r"...Ag29_18112025\Ag29-list.csv")

In [3]:
# From df select only the data that have been processed in PETS2 based on the criteria that cif_pets = YES
processed_df = df[(df['cif_pets'] == 'Yes')].copy()

In [7]:
# Only the processed data, which is stored in processed_df
processed_df.describe()

,Til start,Tilt end,a,b,c,alpha,beta,gamma,Unit cell volume,Resolution,Mosaicity
count,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.00000,15.000000,13.000000,15.000000
mean,41.196667,-23.012667,33.266667,33.266667,33.266667,89.998667,89.999333,89.99800,36834.986000,0.979231,0.085933
std,37.550500,74.797765,0.127541,0.127541,0.127541,0.003519,0.002582,0.00414,426.712625,0.117789,0.061346
min,-58.760000,-59.880000,33.010000,33.010000,33.010000,89.990000,89.990000,89.99000,35980.900000,0.800000,0.000000
25%,44.905000,-59.755000,33.180000,33.180000,33.180000,90.000000,90.000000,90.00000,36545.980000,0.940000,0.055000
50%,59.470000,-58.870000,33.250000,33.250000,33.250000,90.000000,90.000000,90.00000,36772.050000,1.000000,0.084000
75%,59.630000,-42.335000,33.350000,33.350000,33.350000,90.000000,90.000000,90.00000,37113.985000,1.000000,0.112500
max,60.500000,211.190000,33.510000,33.510000,33.510000,90.000000,90.000000,90.00000,37640.460000,1.200000,0.243000


In [9]:
# This part of the program identifies different crystals in the processed data based on how different the lattice parameters
# Set the tolerance level such that if two crystals have the lattice parameter less than the tolerances, they are considered the same crystal
LEN_TOL = 1      # Å
ANG_TOL = 0.5      # degrees


# This function compares the unit cell parameters
def same_crystal(row, ref):
    return (
        abs(row['a'] - ref['a']) < LEN_TOL and
        abs(row['b'] - ref['b']) < LEN_TOL and
        abs(row['c'] - ref['c']) < LEN_TOL and
        abs(row['alpha'] - ref['alpha']) < ANG_TOL and
        abs(row['beta']  - ref['beta'])  < ANG_TOL and
        abs(row['gamma'] - ref['gamma']) < ANG_TOL
    )

# Assign crystal groups and add a new column "crystal_id"
processed_df = processed_df.reset_index(drop=True)
processed_df['crystal_id'] = -1

crystal_counter = 1

for i, row in processed_df.iterrows():
    if processed_df.loc[i, 'crystal_id'] != -1:
        continue

    # assign new crystal id
    processed_df.loc[i, 'crystal_id'] = crystal_counter

    for j in range(i + 1, len(processed_df)):
        if same_crystal(processed_df.loc[j], row):
            processed_df.loc[j, 'crystal_id'] = crystal_counter

    crystal_counter += 1

In [11]:
# Check if the ids are assigned properly
processed_df[['Data name', 'crystal_id', 'a', 'b', 'c','alpha','beta','gamma']]

,Data name,crystal_id,a,b,c,alpha,beta,gamma
0,2,1,33.51,33.51,33.51,90.00,90.00,89.99
1,5,1,33.33,33.33,33.33,90.00,90.00,90.00
2,6,1,33.34,33.34,33.34,89.99,90.00,90.00
3,8,1,33.20,33.20,33.20,90.00,90.00,89.99
4,9,1,33.36,33.36,33.36,89.99,90.00,90.00
5,11,1,33.01,33.01,33.01,90.00,90.00,90.00
6,12,1,33.22,33.22,33.22,90.00,90.00,90.00
7,16,1,33.40,33.40,33.40,90.00,90.00,90.00
8,18,1,33.16,33.16,33.16,90.00,90.00,89.99
9,20,1,33.25,33.25,33.25,90.00,89.99,90.00


In [13]:
processed_df = processed_df.rename(columns={
    'Completness': 'Completeness',
    'CC1\\2': 'CC1/2'
})

In [15]:
# Clean and standardize metrics
quality_cols = [
    'Completeness', 'Resolution', 'Mosaicity',
    'Rint(obs)', 'Rint(all)', 'CC1/2'
]

for col in ['Completeness', 'Rint(obs)', 'Rint(all)', 'CC1/2']:
    processed_df[col] = (
        processed_df[col]
        .astype(str)
        .str.replace('%', '', regex=False)
        .replace('', np.nan)
        .astype(float)
    )
# Normalize each metric
def robust_z(series):
    med = np.nanmedian(series)
    mad = np.nanmedian(np.abs(series - med))
    return (series - med) / (mad if mad != 0 else 1)

# Apply normalization
norm_df = processed_df.copy()

# Higher is better
for col in ['Resolution', 'Completeness', 'CC1/2']:
    norm_df[col + '_z'] = robust_z(norm_df[col])

# Lower is better → invert sign
for col in ['Mosaicity', 'Rint(obs)', 'Rint(all)']:
    norm_df[col + '_z'] = -robust_z(norm_df[col])

# Build a composite quality score
z_cols = [c for c in norm_df.columns if c.endswith('_z')]

norm_df['quality_score'] = norm_df[z_cols].mean(axis=1)

# Rank datasets per crystal
norm_df['quality_rank'] = (
    norm_df
    .groupby('crystal_id')['quality_score']
    .rank(ascending=False, method='dense')
)

# Since some metrics are correlated (e.g. Lower resolution cutoff means better R values), we are weighting resolution higher
# so it dominates the score
weights = {
    'Resolution_z': 0.3,
    'Completeness_z': 0.2,
    'CC1/2_z': 0.2,
    'Rint(obs)_z': 0.1,
    'Rint(all)_z': 0.1,
    'Mosaicity_z': 0.1
}

norm_df['quality_score'] = sum(norm_df[col] * w for col, w in weights.items())

In [17]:
# 1. Keep only rows with valid metrics (including all used in quality score)
quality_metrics = ['Completeness', 'Resolution', 'CC1/2', 'Rint(obs)', 'Rint(all)', 'Mosaicity']
top_df = norm_df.dropna(subset=quality_metrics).copy()

# 2. Recompute quality_rank per crystal so it starts at 1
top_df['quality_rank'] = top_df.groupby('crystal_id')['quality_score'] \
                               .rank(method='dense', ascending=False) \
                               .astype(int)

# 3. Sort and select columns, putting quality_rank first
columns_to_show = ['quality_rank',
                   'Data name',
                   'crystal_id',
                   'quality_score'] + quality_metrics

top_df = top_df.sort_values(['crystal_id', 'quality_rank'])
top_df = top_df[columns_to_show].reset_index(drop=True)

# 4. Display clean table without row numbers
from IPython.display import display, HTML
display(HTML(top_df.to_html(index=False)))


quality_rank,Data name,crystal_id,quality_score,Completeness,Resolution,CC1/2,Rint(obs),Rint(all),Mosaicity
1,18,1,1.306777,95.4,1.20,99.66,13.53,37.69,0.130
2,16,1,0.792416,98.6,1.00,98.88,12.76,34.14,0.000
3,11,1,0.653533,98.9,1.00,99.21,11.73,36.71,0.092
4,4-b,1,0.584802,98.0,1.13,98.85,14.90,73.00,0.131
5,21,1,0.465799,91.7,1.10,98.53,11.18,53.26,0.039
6,20,1,0.450782,98.3,1.00,99.20,14.26,40.23,0.095
7,10+11,1,-0.323749,98.3,1.00,98.67,16.40,47.72,0.243
8,25,1,-0.688861,96.6,0.94,97.98,16.58,49.94,0.149
9,9,1,-0.752944,94.6,0.94,97.77,16.23,58.92,0.084
10,12,1,-1.351696,94.0,0.80,96.74,13.10,31.83,0.052
